In [1]:
# Install necessary libraries
!pip install datasets

# Import libraries
import numpy as np
from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

In [2]:
# Load the dataset (using Hugging Face's wikitext dataset)
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Extract text data
text_data = dataset['train']['text']

# Filter out empty lines
text_data = [text for text in text_data if text.strip() != ""]

# Tokenize the text
tokenizer = Tokenizer(num_words=20000)  # Increase vocabulary size
tokenizer.fit_on_texts(text_data)

# Create input sequences
input_sequences = []
for text in text_data:
    tokenized_text = tokenizer.texts_to_sequences([text])[0]
    for i in range(1, len(tokenized_text)):
        input_sequences.append(tokenized_text[:i+1])

# Limit dataset size to avoid memory issues
input_sequences = input_sequences[:10000]  # Use only 10,000 sequences

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [3]:
# Set a smaller max sequence length
max_len = 50

# Pad sequences
padded_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='pre')

# Prepare X and y
X = padded_sequences[:, :-1]
y = padded_sequences[:, -1]

In [4]:
# Use a data generator to avoid memory issues
class DataGenerator(Sequence):
    def __init__(self, input_sequences, batch_size, max_len, vocab_size):
        self.input_sequences = input_sequences
        self.batch_size = batch_size
        self.max_len = max_len
        self.vocab_size = vocab_size

    def __len__(self):
        return int(np.ceil(len(self.input_sequences) / self.batch_size))

    def __getitem__(self, idx):
        batch_sequences = self.input_sequences[idx * self.batch_size:(idx + 1) * self.batch_size]
        padded_sequences = pad_sequences(batch_sequences, maxlen=self.max_len, padding='pre')
        X = padded_sequences[:, :-1]
        y = padded_sequences[:, -1]
        return X, y

In [5]:
# Create the generator
batch_size = 32
vocab_size = len(tokenizer.word_index) + 1
data_gen = DataGenerator(input_sequences, batch_size, max_len, vocab_size)


In [6]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_len-1))
model.add(LSTM(256, return_sequences=True))  # Increase units
model.add(LSTM(256))  # Add another LSTM layer
model.add(Dense(vocab_size, activation='softmax'))

# Compile the model with sparse_categorical_crossentropy
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(data_gen, epochs=50, callbacks=[early_stopping])  # Train for more epochs

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 31ms/step - accuracy: 0.0785 - loss: 8.8154
Epoch 2/50
  5/313 ━━━━━━━━━━━━━━━━━━━━ 9s 30ms/step - accuracy: 0.1066 - loss: 6.4152

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.0795 - loss: 6.6740
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.0793 - loss: 6.5678
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.0836 - loss: 6.3106
Epoch 5/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.0983 - loss: 6.1091
Epoch 6/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.0993 - loss: 5.9514
Epoch 7/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.1086 - loss: 5.7827
Epoch 8/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.1174 - loss: 5.6274
Epoch 9/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.1240 - loss: 5.4957
Epoch 10/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.1321 - loss: 5.3666
Epoch 11/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.1366 - loss: 5.2147
Epoch 12/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.1396 - loss: 5.1120
Epoch 13/50
313/313 ━━━━━━━━━━━━━━━━━━━━

In [7]:
# Temperature sampling function
def generate_text_with_temperature(model, tokenizer, seed_text, max_len=50, temperature=1.0):
    for _ in range(max_len):
        tokenized_text = tokenizer.texts_to_sequences([seed_text])[0]
        padded_text = pad_sequences([tokenized_text], maxlen=max_len-1, padding='pre')
        predictions = model.predict(padded_text, verbose=0)[0]

        # Apply temperature sampling
        predictions = np.log(predictions) / temperature
        exp_predictions = np.exp(predictions)
        predictions = exp_predictions / np.sum(exp_predictions)

        predicted_word_idx = np.random.choice(len(predictions), p=predictions)
        predicted_word = tokenizer.index_word.get(predicted_word_idx, '')
        seed_text += " " + predicted_word
    return seed_text

In [11]:
# Test with temperature sampling
seed_text = "how are"
print(generate_text_with_temperature(model, tokenizer, seed_text, temperature=0.7))

how are launched and tgs 000 if the situations deemed the blitz system is carried over directly received barker 's children do though barker 's football in a trunk in two forces and her arms that he serialized in dengeki february 1973 aged 77 years two funeral services were held were calamity
